<a href="https://colab.research.google.com/github/Devasy23/mrp/blob/main/Decoder_Encoder1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install opencv-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import cv2
import numpy as np
import struct
import os
import zlib

In [5]:
zip_file_path = 'input.zip'
video_file_path = 'video.mp4'


In [6]:
with open(zip_file_path, 'rb') as f:
    binary_data = f.read()


In [9]:
compressed_data = zlib.compress(binary_data)



In [10]:
cap = cv2.VideoCapture(video_file_path)

frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    frames.append(frame)

cap.release()


In [11]:
chunk_size = 100  # adjust this value based on the size of the compressed data and the resolution of the video

chunks = [compressed_data[i:i+chunk_size] for i in range(0, len(compressed_data), chunk_size)]


In [12]:
for i, chunk in enumerate(chunks):
    binary_str = ''.join(format(byte, '08b') for byte in chunk)  # convert the binary data to a string of 0s and 1s
    binary_list = [int(bit) for bit in binary_str]  # convert the string to a list of integers

    frame = frames[i % len(frames)]  # repeat the frames if the number of chunks exceeds the number of frames

    for j, pixel in enumerate(frame.reshape(-1, 3)):
        if j >= len(binary_list):
            break

        pixel[2] = (pixel[2] & 0b11111110) | binary_list[j]  # modify the least significant bit of the red channel


ZeroDivisionError: ignored

In [14]:
import cv2
import zipfile
import numpy as np
# gpt4
def encode_zip_to_video(zip_filename, video_filename, width=1920, height=1080):
    with zipfile.ZipFile(zip_filename, 'r') as zfile:
        zip_data = zfile.read(zfile.namelist()[0])

    bitstream = [int(b) for byte in zip_data for b in format(byte, '08b')]

    frame_size = width * height
    num_frames = (len(bitstream) + frame_size - 1) // frame_size

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video_writer = cv2.VideoWriter(video_filename, fourcc, 30, (width, height), isColor=False)

    for i in range(num_frames):
        frame_data = bitstream[i * frame_size:(i + 1) * frame_size]
        frame_data += [0] * (frame_size - len(frame_data))

        frame = np.array(frame_data, dtype=np.uint8).reshape((height, width))
        video_writer.write(frame)

    video_writer.release()

if __name__ == '__main__':
    encode_zip_to_video('input.zip', 'output.mp4')


In [19]:
# bard
import cv2
import zipfile
import numpy as np

def encode_zip_to_video(zip_filename, video_filename, width=1920, height=1080, frame_rate=30, color_mode='rgb'):
    if not os.path.exists(zip_filename):
        raise FileNotFoundError(f'The zip file {zip_filename} does not exist.')

    if os.path.exists(video_filename):
        raise FileExistsError(f'The video file {video_filename} already exists.')

    with zipfile.ZipFile(zip_filename, 'r') as zfile:
        zip_data = zfile.read(zfile.namelist()[0])

    bitstream = [int(b) for byte in zip_data for b in format(byte, '08b')]

    frame_size = width * height * 3 if color_mode == 'rgb' else width * height
    num_frames = (len(bitstream) + frame_size - 1) // frame_size

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video_writer = cv2.VideoWriter(video_filename, fourcc, frame_rate, (width, height), color_mode == 'rgb')

    for i in range(num_frames):
        frame_data = bitstream[i * frame_size:(i + 1) * frame_size]
        frame_data += [0] * (frame_size - len(frame_data))

        frame = np.array(frame_data, dtype=np.uint8).reshape((height, width, 3 if color_mode == 'rgb' else 1))
        video_writer.write(frame)

    video_writer.release()

if __name__ == '__main__':
    encode_zip_to_video('input.zip', 'output.mp4')

In [20]:
#  bard 2
import cv2
import zipfile
import numpy as np

def encode_zip_to_video(zip_filename, video_filename, width=1920, height=1080):
    with zipfile.ZipFile(zip_filename, 'r') as zfile:
        zip_data = zfile.read(zfile.namelist()[0])

    bitstream = [int(b) for byte in zip_data for b in format(byte, '08b')]

    frame_size = width * height
    num_frames = (len(bitstream) + frame_size - 1) // frame_size

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video_writer = cv2.VideoWriter(video_filename, fourcc, 30, (width, height), isColor=False)

    for i in range(num_frames):
        frame_data = bitstream[i * frame_size:(i + 1) * frame_size]
        frame_data += [0] * (frame_size - len(frame_data))

        frame = np.array(frame_data, dtype=np.uint8).reshape((height, width))
        video_writer.write(frame)

    video_writer.release()

if __name__ == '__main__':
    encode_zip_to_video('input.zip', 'output.mp4')


In [16]:
import cv2
import numpy as np

def decode_video_to_zip(video_filename, output_zip_filename):
    video_capture = cv2.VideoCapture(video_filename)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = width * height

    bitstream = []

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_bits = frame.flatten().tolist()
        bitstream.extend(frame_bits)

    video_capture.release()

    # Remove the padding zeroes, if any
    # bitstream = bitstream[:bitstream.index(1)]

    # Convert the bitstream back to bytes
    byte_data = bytearray()
    for i in range(0, len(bitstream), 8):
        byte = int(''.join(map(str, bitstream[i:i + 8])), 2)
        byte_data.append(byte)

    # Save the byte data to a zip file
    with open(output_zip_filename, 'wb') as output_zip_file:
        output_zip_file.write(byte_data)

if __name__ == '__main__':
    decode_video_to_zip('output.mp4', 'decoded_output.zip')


In [ ]:
# bard
import cv2
import numpy as np

def decode_video_to_zip(video_filename, output_zip_filename):
    video_capture = cv2.VideoCapture(video_filename)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_size = width * height

    bitstream = []

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_bits = frame.flatten().tolist()
        bitstream.extend(frame_bits)

    video_capture.release()

    # Remove the padding zeroes, if any
    for i in range(len(bitstream) - 1, -1, -1):
        if bitstream[i] == 0:
            bitstream.pop(i)

    # Convert the bitstream back to bytes
    byte_data = bytearray()
    for i in range(0, len(bitstream), 8):
        byte = int(''.join(map(str, bitstream[i:i + 8])), 2)
        byte_data.append(byte)

    # Save the byte data to a zip file
    with zipfile.ZipFile(output_zip_filename, 'w') as zfile:
        zfile.writestr(zfile.namelist()[0], bytes(byte_data))

if __name__ == '__main__':
    decode_video_to_zip('output.mp4', 'decoded_output.zip')
